# Sequence save poses

Save a list of poses for an entire sequence for use with full Bundle Adjustment

In [1]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
os.environ['STEREO_CONFIDENCE'] = '1.6'
os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from pyurb.urb_kitti import *
%matplotlib inline
import numpy as np
import glob

Read the first 10 frames from KITTI sequence 00.

In [2]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/01/image_2'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/01/image_3'

frame = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 4.55 ms, sys: 922 µs, total: 5.47 ms
Wall time: 4.65 ms


Inspect the FramePoints for the first frame

We can also manually compute the Depth of the points and show these

Turn the list of frames into a sequence. The first frame is automatically taken as the KeyFrame and its FramePoints are rerieved, depth is estimated, only the most confident points are kept, and they are tracked in the other frames.

In [3]:
seq = Sequence()
for f in ProgressBar()(frame[:25]):
    seq.add_frame(f)

 20% (5 of 25) |#####                     | Elapsed Time: 0:00:08 ETA:  0:00:35

invalid speed 4.20373948795 -1.03510464781 
 [[ 0.97477079  0.0036529  -0.22317834  1.76321253]
 [-0.00251859  0.99998243  0.00536694 -0.1015843 ]
 [ 0.22319402 -0.00466944  0.97476286 -0.01759371]
 [ 0.          0.          0.          1.        ]]


 64% (16 of 25) |################         | Elapsed Time: 0:00:23 ETA:  0:00:07

150 -1.00338851616 [[  9.98943459e-01   9.72309543e-05  -4.59560289e-02  -2.08941232e-02]
 [ -1.12544650e-05   9.99998249e-01   1.87109748e-03   7.17438652e-03]
 [  4.59561304e-02  -1.86860338e-03   9.98941711e-01  -1.00338852e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
invalid speed -2.08640786322 -1.09959396519 
 [[  9.95641204e-01   3.08054441e-03  -9.32153570e-02   6.39715736e-02]
 [ -2.89525640e-03   9.99993555e-01   2.12291185e-03   3.22341005e-02]
 [  9.32212960e-02  -1.84377615e-03   9.95643707e-01  -2.08640786e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


 68% (17 of 25) |#################        | Elapsed Time: 0:00:26 ETA:  0:00:08

146 -1.00517670237 [[  9.98912562e-01   3.57953037e-03  -4.64852676e-02  -1.64516709e-02]
 [ -3.59418415e-03   9.99993514e-01  -2.31654796e-04   3.21980523e-02]
 [  4.64841369e-02   3.98479497e-04   9.98918949e-01  -1.00517670e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
invalid speed -2.04236292523 -1.06812154425 
 [[  9.95616080e-01   3.06044939e-03  -9.34839845e-02   8.86356017e-02]
 [ -2.99482780e-03   9.99995161e-01   8.42239115e-04   3.24173737e-02]
 [  9.34861098e-02  -5.58578370e-04   9.95620427e-01  -2.04236293e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


100% (25 of 25) |#########################| Elapsed Time: 0:00:34 Time: 0:00:34


In [6]:
import shutil
OUTDIR = 'results'
try:
    shutil.rmtree(OUTDIR)
except:
    pass
os.mkdir(OUTDIR)
keyframes_np = keyframes_to_np(seq.keyframes)
mappoints = get_mappoints(seq.keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)
suffix = '_{}_{}_{}_{}'.format(FRAMECOUNT, PATCH_SIZE, STEREO_CONFIDENCE, SEQUENCE_CONFIDENCE)
np.save(OUTDIR + '/mappoints' + suffix, mappoints_np)
np.save(OUTDIR + '/links' + suffix, links_np)
np.save(OUTDIR + '/keyframes' + suffix, keyframes_np)